In [12]:
import dash
from dash import dcc, html
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import numpy as np


In [13]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [14]:
df = movies.merge(credits, on='title')

In [15]:
print(df[['title', 'budget', 'revenue', 'cast', 'crew']].head())

                                      title     budget     revenue  \
0                                    Avatar  237000000  2787965087   
1  Pirates of the Caribbean: At World's End  300000000   961000000   
2                                   Spectre  245000000   880674609   
3                     The Dark Knight Rises  250000000  1084939099   
4                               John Carter  260000000   284139100   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52f

In [17]:
# Convert stringified JSON to Python list/dict
import pandas as pd
import numpy as np
import json  

# Then continue loading and working with your data...

def get_director(crew_str):
    crew = json.loads(crew_str)
    for person in crew:
        if person['job'] == 'Director':
            return person['name']
    return np.nan

def get_top_cast(cast_str):
    cast = json.loads(cast_str)
    return [person['name'] for person in cast[:3]]

df['director'] = df['crew'].apply(get_director)
df['top_cast'] = df['cast'].apply(get_top_cast)

# Keep the main features
df_model = df[['budget', 'popularity', 'runtime', 'vote_average', 'vote_count', 'revenue', 'director', 'top_cast']].copy()

# Fill missing numeric values
df_model.fillna({'budget': df_model['budget'].mean(), 'runtime': df_model['runtime'].mean()}, inplace=True)

# One-hot encode director (top 10 only)
top_directors = df_model['director'].value_counts().head(10).index
df_model['director'] = df_model['director'].apply(lambda x: x if x in top_directors else 'Other')
df_model = pd.get_dummies(df_model, columns=['director'], drop_first=True)

# Convert top cast list to individual columns
df_model[['actor_1', 'actor_2', 'actor_3']] = pd.DataFrame(df_model['top_cast'].tolist(), index=df_model.index)
df_model.drop(columns='top_cast', inplace=True)

# One-hot encode actors (optional, keep top 10 for each)
for i in ['actor_1', 'actor_2', 'actor_3']:
    top_actors = df_model[i].value_counts().head(10).index
    df_model[i] = df_model[i].apply(lambda x: x if x in top_actors else 'Other')

df_model = pd.get_dummies(df_model, columns=['actor_1', 'actor_2', 'actor_3'], drop_first=True)


In [18]:
# Define features and target
X = df_model.drop(columns='revenue')
y = df_model['revenue']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = XGBRegressor()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)


RMSE: 89338200.69733144


In [20]:
import dash
from dash import html, dcc
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# Load datasets
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
df = movies.merge(credits, on='title')

import json

def get_director(crew_str):
    try:
        crew = json.loads(crew_str)
        for person in crew:
            if person['job'] == 'Director':
                return person['name']
    except:
        return np.nan

def get_top_cast(cast_str):
    try:
        cast = json.loads(cast_str)
        return [person['name'] for person in cast[:3]]
    except:
        return []

df['director'] = df['crew'].apply(get_director)
df['top_cast'] = df['cast'].apply(get_top_cast)

# Feature engineering
df_model = df[['budget', 'popularity', 'runtime', 'vote_average', 'vote_count', 'revenue', 'director', 'top_cast']].copy()
df_model.fillna({'budget': df_model['budget'].mean(), 'runtime': df_model['runtime'].mean()}, inplace=True)

top_directors = df_model['director'].value_counts().head(10).index
df_model['director'] = df_model['director'].apply(lambda x: x if x in top_directors else 'Other')
df_model = pd.get_dummies(df_model, columns=['director'], drop_first=True)

df_model[['actor_1', 'actor_2', 'actor_3']] = pd.DataFrame(df_model['top_cast'].tolist(), index=df_model.index)
df_model.drop(columns='top_cast', inplace=True)

for actor_col in ['actor_1', 'actor_2', 'actor_3']:
    top_actors = df_model[actor_col].value_counts().head(10).index
    df_model[actor_col] = df_model[actor_col].apply(lambda x: x if x in top_actors else 'Other')

df_model = pd.get_dummies(df_model, columns=['actor_1', 'actor_2', 'actor_3'], drop_first=True)

# Train model
X = df_model.drop(columns='revenue')
y = df_model['revenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Sample visualization data
sample = pd.DataFrame({
    'Movie': df.iloc[y_test.index[:10]]['title'].values,
    'Actual Revenue': y_test[:10].values,
    'Predicted Revenue': y_pred[:10]
})

fig = px.bar(sample, x='Movie', y=['Actual Revenue', 'Predicted Revenue'],
             barmode='group', title='Actual vs Predicted Revenue for Sample Movies')

# Dash app
app = dash.Dash(__name__)
app.title = "Movie Revenue Dashboard"

app.layout = html.Div([
    html.H1("🎬 Movie Revenue Prediction Dashboard", style={
        'textAlign': 'center',
        'color': '#2c3e50',
        'fontSize': '40px',
        'marginBottom': '10px'
    }),
    html.H2("📊 Model Evaluation", style={
        'textAlign': 'center',
        'color': '#34495e',
        'fontSize': '28px',
        'marginBottom': '20px'
    }),
    html.P(f"Root Mean Squared Error on Test Set: ${round(rmse, 2):,}", style={
        'textAlign': 'center',
        'fontSize': '22px',
        'color': '#16a085',
        'marginBottom': '40px'
    }),
    dcc.Graph(figure=fig)
], style={'fontFamily': 'Arial, sans-serif', 'padding': '20px'})

if __name__ == '__main__':
    app.run(debug=True)
